

<br>

<div style ="width: 100%;">
<table style ="width: 100%;background-color:float:center;">
<tr>

</tr>
    
<tr>
        
<td>
<div style="font-size: 21px;Margin-left:-10px;">
<b>Sistema criptográfico de chave pública de Rabin<b>
</div>
    <br>
    <br>
<div style="font-size: 16px;">
    
Realizado por:

Carlos Ferreira a87953

Daniel Ribeiro a87994

João Nogueira a87973
</div>
</td>
<td >
<div style="display:inline-block; float:right;Margin-right:0px;Margin-top:20px;">
<img src="https://www.ecum.uminho.pt/SiteAssets/ECUM-PT.png" width="260" height="120">
<div>
</td>
</tr>
</table>
</div>
    

# Sistema de Rabin
No seguinte trabalho foi implementado o sistema de Rabin, que consiste em uma técnica para a encriptação e desicriptação de mensagens, visando a proteção da mesma. 
<br>
<br>
De seguida iremos explicar como foi implementado este sistema em python, explicando cada função do mesmo suportado pela teoria por de trás disto tudo.

## Geração das chaves

Sistema de Rabin é um criptossistema assimétrico, onde cada pessoa possui uma chave pública, que pode ser partilhada e publicada, e possui também uma chave privada que deverá ser conhecida apenas pelo proprietário da mesma.
Num criptossistema assimétrico, qualquer pessoa pode encriptar uma mensagem usando a chave pública do destinatário. Essa mensagem encriptada deve ser desencriptada com a chave privada do destinatário.

<img style= "display: block;margin-left: auto;margin-right: auto;" src="https://cdn.discordapp.com/attachments/821737859592880141/834488578339635240/1024px-Public_key_encryption.png" width="480" height="270">

Começamos por gerar dois números primos grandes, distintos p e q. De forma a simplificar os cálculos das raízes quadradas módulo p e q, escolhemos p e q tal que:
$${p\equiv q\equiv 3 \space ({\bmod {4}}})$$
Mas o esquema funciona com qualquer primos.

Calculámos de seguida:
$${\displaystyle n=p\cdot q}$$ 
Este $\textbf{n}$ será a $\textbf{chave pública}$ enquanto que a $\textbf{chave privada}$ será o par $\textbf{(p,q)}$.



Criámos então uma função $\textbf{gera_primo}$ que nos vai auxiliar na geração de primos grandes congruentes com 3 módulo 4.

In [2]:
def gera_primo():
    
    p = random_prime(2^100,2^105)
    
    while (mod(p,4) != 3):
        p = random_prime(2^100,2^105)
    
    return p

A nossa função $\textbf{key_generation}$ recebe como argumento uma lista de nomes, e para cada nome dessa lista vão ser gerados dois primos p e q pela $\textbf{gera_primo}$  cujo par (p,q) vai formar a $\textbf{chave privada}$ relativa a esse nome, e será também calculado o $\textbf{n}$, que será a $\textbf{chave pública}$ relativa a esse nome.
Ambas as chaves serão depois adicionadas ao dicionário.

In [3]:
#Função de geração de chaves públicas e privadas para o sender e para o receiver
def key_generation (nomes):

    keys = {}
    for nome in nomes:
        
        p = gera_primo()
        q = gera_primo()
        n = p * q


        keys[nome] = {}
        keys[nome]['public_key'] = n
        keys[nome]['private_key'] = (p,q)

    return keys

## Encriptação

Aqui temos a função de encriptação da mensagem que recebe o dicionário das chaves e o receptor da mesma.
<br>
<br> 
Começamos por ir buscar a chave pública do receptor, e geramos uma mensagem que tem até metade do comprimento dessa chave, pois vamos depois duplicar-la de seguida (para a codificar) e esta duplicação não pode exceder o intervalo [0, n-1] sendo n o tamanho da chave pública.
<br> 
<br> 
Após isso é só usar o algoritmo da encriptação sendo a nossa mensagem cifrada (c), a mensagem gerada elevada a 2 em módulo n:
<br> 
<br> 
$$
c = m^{2} \space mod \space n
$$



In [4]:
def encriptar (keys , receptor):

    #Vamos buscar a chave pública do receptor
    n = keys[receptor]['public_key']
           

    mens = randint(0,int(str(n)[0:(len(str(n))//(2))]))  #pega só na primeira metade da chave
    print("mens = ",mens)
    mm = str(mens) * 2                                   #duplica a chave
    mens = int(mm)

    c = power_mod(mens , 2 , n)           
    print ("c = " , c)
    return c

## Desencriptação

E finalmente temos a função de desencriptação que há semelhança da função de encriptação recebe o dicionário das chaves e o receptor da mensagem e também o c que é a mensagem encriptada.
<br> 
<br> 
Guardamos em $\textbf{p}$ $\textbf{q}$ os primos da chave privada e em $\textbf{n}$ a chave pública.
<br> 
Após isso aplicamos o algoritmo de Rabin.
<br> 
Começando por usar o algoritmo de euclides extendido para calcular o a e b que satisfaça:
$$
    a*p + b*q = 1
$$
<br> 
Com $\textbf{a}$ e $\textbf{b}$ calculados já podemos calcular o nosso $\textbf{r}$, $\textbf{s}$, $\textbf{x}$ e $\textbf{y}$ apenas usando as seguintes fórmulas:
$$
    r = c^{(p+1)/4} \space mod \space p\\
    s = c^{(q+1)/4} \space mod \space q\\
    x = a*p*s + b*q*r \space mod \space n \\
    y = a*p*s - b*q*r \space mod \space n \\
$$

Assim sendo as 4 alternativas da mensagem serão:

$$
    x \\
    -x \space mod \space n \\
    y \\
    -y \space mod \space n \\
$$



In [9]:
def desencriptar (keys , c , receptor):

    p = keys[receptor]['private_key'][0]
    q = keys[receptor]['private_key'][1]
    n = keys[receptor]['public_key']

    
    _,a,b = xgcd(p,q)  #algoritmo de euclides extendido 

    #aplicação das fórmulas
    r = power_mod(c , (p+1)//4 , p)
    s = power_mod(c, (q+1)//4 , q)
    x = mod((a*p*s) + (b*q*r) , n )
    y = mod((a*p*s) - (b*q*r) , n )


    #impressão das possíveis mensagens
    print("m1 = ",x,"\nm2 = ",mod(-x,n),"\nm3 = ",y,"\nm4 = ",mod(-y,n),"\n")    
    
    
    

##  Premissa de segurança

Uma desvantagem do esquema de chave pública de Rabin é que o receptor se depara com a tarefa de selecionar a mensagem correta entre quatro possibilidades. Esta ambiguidade pode ser facilmente superada na prática, adicionando redundância pré-especificada para o texto simples original antes da criptografia, como no nosso caso a repetição da mensagem.  Então, com alta probabilidade, exatamente uma das quatro raízes quadradas m1, m2, m3, m4 de um texto cifrado legítimo c possuirão essa redundância, e o receptor irá selecioná-lo como a mensagem pretendida. Se nenhuma das raízes quadradas de c possui essa redundância, então o receptor deve rejeitar a mensagem cifrada como fraudulenta. 



Se a redundância for usada como acima, o esquema de Rabin não é mais suscetível ao $\bf{"chosen \space ciphertext \space attack"}$. 

Num $\bf{"chosen \space ciphertext \space attack"}$ assume-se que o atacante de alguma forma tem acesso as desencriptações das mensagens que a pessoa A recebe, mas não sabe a chave privada de A e tem como objetivo descobri-la. 

 Se o atacante intercetar uma mensagem m com a redundância necessária , mesmo com acesso a desencriptação da mensagem m, o atacante não saberá a mensagem real que foi enviada. Por outro lado, se o atacante intercetar uma mensagem m que não contém a redundância necessária, então, com alta probabilidade, nenhuma das quatro raízes quadradas possuirá a redundância necessária logo o atacante não obterá resposta.

## Exemplos
De seguida serão mostrados do bom funcionamento da nossa implementação do algoritmo de Rabin.

In [10]:
#função auxiliar para impressão bonita do dicionário
def prettyPrint(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            prettyPrint(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))



keys = key_generation(['Henrique','Daniel','João'])
prettyPrint (keys)

Henrique
	public_key
		100217162133140500481816006495814265940501603953580088575197
	private_key
		(201522624413678108220030535403, 497299806533972370748086010199)
Daniel
	public_key
		6583065895299005005845626962612827406188814347697960039321
	private_key
		(23004552433753969758733665347, 286163615408567870474842880243)
João
	public_key
		597346852419103500337442533320865481361845780218927241044489
	private_key
		(565947427761523718579409587723, 1055481168598597685533249478843)


In [13]:
# Henrique -> Daniel
# Henrique envia uma mensagem ao Daniel
print("----------------------------------------------------------------------------")
print("Henrique -> Daniel")
print("----------------------------------------------------------------------------")
desencriptar(keys ,encriptar(keys , 'Daniel') , 'Daniel')


----------------------------------------------------------------------------
Henrique -> Daniel
----------------------------------------------------------------------------
mens =  44089348291439183609140894863
c =  3707000897263643404956909504280340108608312351057881423903
m1 =  4408934829143918360914089486344089348291439183609140894863 
m2 =  2174131066155086644931537476268738057897375164088819144458 
m3 =  3101406365640600648418856481355197112482325685325473425056 
m4 =  3481659529658404357426770481257630293706488662372486614265 



In [14]:
# João -> Henrique
# João envia uma mensagem ao Henrique
print("----------------------------------------------------------------------------")
print("João -> Henrique")
print("----------------------------------------------------------------------------")
desencriptar(keys ,encriptar(keys , 'Henrique') , 'Henrique')

----------------------------------------------------------------------------
João -> Henrique
----------------------------------------------------------------------------
mens =  80978290608524098872894401860
c =  28059295607025097329468668327164743055833547120897888885304
m1 =  21236921514213759191107449016586192752936407939416001199450 
m2 =  78980240618926741290708557479228073187565196014164087375747 
m3 =  92119333072288090594526566309733287649893079854707194173337 
m4 =  8097829060852409887289440186080978290608524098872894401860 

